# General practitioner referral

This document shows an example of evolution of a prompt from a simple text to a full prompt with a workflow. It uses Semantic Kernel.

Goal: Improve the referral workflow of a GP practice, to automate documentation generation and delivery.
Scenario: A Dr. Smith sees a patient, 80 year old man, Fred, at his practice. Fred has broken his upper elbow by falling off his ebike. He wants to refer Fred to the Dr. Jones, at the Fracture Clinic in Contoso Hospital.

Inputs:
1. Medical short hand Dr. Smith creating during the visit:

```text
S: Pt c/o pain in R elbow after fall. Hx of HTN, T2DM. O: R elbow swollen, tender to palpation. ROM limited. No obvious deformity. A: Suspected R upper elbow fx. P: X-ray ordered. RICE, analgesics, sling for immobilization. Ref to Fracture Clinic.
```

1. Transcription from the visit.

```text
I saw Fred today. He tells me he fell off his ebike and landed heavily on his elbow. I've sent him to get X-rays and will refer him to Fracture Clinic at Contoso Health.
```

In [33]:
#r "nuget: Microsoft.SemanticKernel, 0.12.207.1-preview"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.SemanticFunctions;

IKernel kernel = KernelBuilder.Create();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
if (useAzureOpenAI) {
    kernel.Config.AddAzureTextCompletionService("davinci", model, azureEndpoint, apiKey);
    kernel.Config.AddAzureChatCompletionService("chatgpt", "gpt-35-turbo", azureEndpoint, apiKey);
}
else
    kernel.Config.AddOpenAITextCompletionService("davinci", model, apiKey, orgId);
    kernel.Config.AddOpenAIChatCompletionService("chatgpt", "gpt-35-turbo", apiKey, orgId);

Installed Packages Microsoft.SemanticKernel, 0.12.207.1-preview

Let's create a semantic function used to explain medical shorthand:

In [34]:
string explainMedicalShorthand = """
Explain the following medical shorthand and what it means.

### Medical shorthand
{{$input}}
""";

Let's configure the prompt, e.g. allowing for some creativity and a sufficient number of tokens.

In [35]:
var promptConfig = new PromptTemplateConfig
{
    Completion =
    {
        MaxTokens = 2000,
        Temperature = 0.0,
        TopP = 0.5,
    }
};

The following code prepares an instance of the template, passing in the TXT and configuration above, 
and a couple of other parameters (how to render the TXT and how the template can access other functions).

In [36]:
var explainMedicalShorthandTemplate = new PromptTemplate(
    explainMedicalShorthand,       // Prompt template defined in natural language
    promptConfig,                  // Prompt configuration
    kernel                         // SK instance
);

Let's transform the prompt template into a function that the kernel can execute:

In [37]:
var explainMedicalShorthandFunctionConfig = new SemanticFunctionConfig(promptConfig, explainMedicalShorthandTemplate);

var explainMedicalShorthandFunction = kernel.RegisterSemanticFunction("ExplainMedicalShorthand", "ExplainMedicalShorthand", explainMedicalShorthandFunctionConfig);

Set up some medical shorthand to explain. The medical shorthand is from a patient visit from a upper elbow fracture event.

In [38]:
var medicalShorthand = """
S: Pt c/o pain in R elbow after fall. Hx of HTN, T2DM. O: R elbow swollen, tender to palpation. ROM limited. No obvious deformity. A: Suspected R upper elbow fx. P: X-ray ordered. RICE, analgesics, sling for immobilization. Ref to Fracture Clinic.
""";

...and run the summary function:

In [39]:
var explainedMedicalShorthand = await kernel.RunAsync(medicalShorthand, explainMedicalShorthandFunction);

Console.WriteLine(explainedMedicalShorthand);



S: Patient complains of pain in right elbow after fall. History of hypertension and type 2 diabetes.

O: Right elbow is swollen and tender to palpation. Range of motion is limited. No obvious deformity.

A: Suspected right upper elbow fracture.

P: X-ray ordered. Rest, Ice, Compression, Elevation, analgesics, and sling for immobilization. Referral to Fracture Clinic.
